In [3]:
!pip install pydub

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install midi2audio

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install pretty_midi

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 2.2 MB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 KB 2.8 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592305 sha256=d2f319ad6594876475dd97cad3ee5456285e90734d81fd5ac16d84cc10e56b25
  Stored in directory: /home/h_b_a_e_/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi


In [6]:
import time
import numpy
from midi2audio import FluidSynth
import pretty_midi
import pandas as pd
from pydub import AudioSegment
import subprocess
import mido

/home/h_b_a_e_/.local/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [10]:
def reverse_midi(name, shift):
    midi_data = pretty_midi.PrettyMIDI(name + '.mid')
    notes_by_instruments = dict()
    instrument_number = 0
    instrument_types = dict()
    min = 127
    max = 0
    for instrument in midi_data.instruments:
        instrument_types[instrument_number] = instrument.program
        partiture = []
        for note in instrument.notes:
            partiture.append([note.pitch, note.start, note.end, note.velocity])
            if max < note.pitch:
                max = note.pitch
            if min > note.pitch:
                min = note.pitch 
        notes_by_instruments[instrument_number] = partiture
        instrument_number += 1
    
    reversed_midi = pretty_midi.PrettyMIDI()
    for instrument_number in instrument_types:
        partiture = pretty_midi.Instrument(program=instrument_types[instrument_number])
        for row in notes_by_instruments[instrument_number]:
            note = pretty_midi.Note(
                velocity=int(row[3]),
                pitch=int(max+min + shift-row[0]),
                start=float(row[1]),
                end=float(row[2])
            )
            partiture.notes.append(note)
        reversed_midi.instruments.append(partiture)
        
    reversed_midi.write(name+'Reversed.mid')
    print('done!')

    
# сюда вводить название MIDI и сдвиг. Сдвиг можно регулировать чтобы отражение звучало комфортно
song_name = 'ThemeB'
reverse_midi(song_name, 16)

done!


In [8]:
# Конвертация
fs = FluidSynth('weedsgm3.sf2')
fs.midi_to_audio('BakamitaiReversed.mid', 'BakamitaiReversed.wav')


FileNotFoundError: [Errno 2] No such file or directory: 'fluidsynth'

In [10]:
AudioSegment.from_wav('BakamitaiReversed.wav').export('BakamitaiReversed.mp3', format="mp3")
print("Конвертация завершена!")

FileNotFoundError: [WinError 2] Не удается найти указанный файл